In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [8]:
num_epochs=8
batch_size=64
learning_rate=0.001

In [5]:
transform=transforms.Compose([transforms.Pad(4),transforms.RandomHorizontalFlip(),transforms.RandomCrop(32),transforms.ToTensor()])

In [14]:
train_dataset=torchvision.datasets.CIFAR10('../../data',train=True,transform=transform,download=False)
test_dataset=torchvision.datasets.CIFAR10("../../data",train=False,transform=transform,download=False)
train_loader=torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,shuffle=True,drop_last=True)
test_loader=torch.utils.data.DataLoader(test_dataset,batch_size=batch_size,shuffle=False)

In [99]:
def conv3x3(in_channels,out_channels,stride=1):
    return nn.Conv2d(in_channels,out_channels,kernel_size=3,stride=stride,padding=1,bias=False)

In [126]:
class ResidualBlock(nn.Module):
    def __init__(self,in_channels,out_channels,stride=1):
        super().__init__()
        self.conv1=conv3x3(in_channels,out_channels,stride)
        self.bn1=nn.BatchNorm2d(out_channels)
        self.relu=nn.ReLU()
        self.conv2=conv3x3(out_channels,out_channels)
        self.bn2=nn.BatchNorm2d(out_channels)
        self.stride=stride
        self.in_channels=in_channels
        self.out_channels=out_channels
        
    def forward(self,x):
        residual=x
#         print("x,shape:",x.shape)
        out=self.conv1(x)
        out=self.bn1(out)
        out=self.relu(out)
#         print(out.shape)
        out=self.conv2(out)
        out=self.bn2(out)
#         print(out.shape)
        if (self.stride != 1) or (self.in_channels != self.out_channels):
            residual_model=nn.Sequential(conv3x3(self.in_channels,self.out_channels,stride=self.stride),
                                   nn.BatchNorm2d(self.out_channels))
            residual=residual_model(x)
#         print("residual.shape:",residual.shape)
        out+=residual
        out=self.relu(out)
        return out

In [147]:
class ResNet(nn.Module):
    def __init__(self,block,layers,num_classes=10):
        super().__init__()
        self.conv=conv3x3(3,16)
        self.bn=nn.BatchNorm2d(16)
        self.relu=nn.ReLU()
        self.layer1=self.make_layer(block,16,16,layers[0])
        self.layer2=self.make_layer(block,16,32,layers[1],2)
        self.layer3=self.make_layer(block,32,64,layers[2],2)
        self.avg_pool=nn.AvgPool2d(8)
        self.fc=nn.Linear(64,num_classes)
    
    def make_layer(self,block,in_channels,out_channels,blocks,stride=1):
        layers=[]
        layers.append(block(in_channels,out_channels,stride))
        for i in range(1,blocks):
            layers.append(block(out_channels,out_channels))
        return nn.Sequential(*layers)
    
    def forward(self,x):
        out=self.conv(x)
        out=self.bn(out)
        out=self.relu(out)
        out=self.layer1(out)
#         print(out.shape)
        out=self.layer2(out)
#         print(out.shape)
        out=self.layer3(out)
#         print(out.shape)
        out=self.avg_pool(out)
        out=out.view(out.size(0),-1)
#         print(out.shape)
        out=self.fc(out)
        return out

In [148]:
model=ResNet(ResidualBlock,[2,2,2])
model.train()

ResNet(
  (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (layer1): Sequential(
    (0): ResidualBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ResidualBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, 

In [149]:
loss_func=nn.CrossEntropyLoss()
optim=torch.optim.Adam(model.parameters(),lr=learning_rate)

In [25]:
def update_lr(optim,lr):
    for param_group in optim.param_groups:
        param_group["lr"]=lr

In [ ]:
print("epoch,\ttotal_step,\tloss")
for epoch in range(num_epochs):
    for i ,(images,labels) in enumerate(train_loader):
        y_hat=model(images)
        loss=loss_func(y_hat,labels)
        optim.zero_grad()
        loss.backward()
        optim.step()
        
        if (i+1) % 100 ==0:
            print("{},\t{},\t{:.4f}".format(epoch+1,i+1,loss.item()))        

epoch,	total_step,	loss
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32

torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 3

torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 3

torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 3

torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 3

...

torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 3

torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 3

torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 3

torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 3

torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 64, 8, 8])
torch.Size([64, 64])
torch.Size([64, 16, 32, 32])
torch.Size([64, 3

In [110]:
w=torch.tensor([[1,1,1],[0,0,0],[0,0,1]])
w=w.view(1,1,3,3)
print(w)

tensor([[[[1, 1, 1],
          [0, 0, 0],
          [0, 0, 1]]]])


In [114]:
x=torch.randn([5,5])
x1=torch.tensor(x>0,dtype=int)
print(x1)
x1=x1.view(1,1,5,5)

tensor([[1, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [1, 0, 0, 1, 1],
        [1, 0, 0, 0, 1],
        [1, 0, 0, 0, 1]])


C:\ProgramData\Anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [115]:
torch.conv2d(x1,w)

tensor([[[[2, 2, 2],
          [1, 1, 2],
          [1, 1, 3]]]])